In [1]:
import socket
import time
from random import choice
from string import ascii_uppercase, digits
import threading
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.applications.vgg19 import VGG19
from scipy.misc import imread, imsave, imshow
import sys
sys.path.append('neural-networks-and-deep-learning/src')
from sys import argv, stderr
import os
from os import listdir
from os.path import isdir, isfile, join
import network2
import copy
from matplotlib import pyplot as plt
from utils import *
from car import Car
import dlib
from skimage import io
import openface

Using TensorFlow backend.


In [7]:
print('Loading Faces haar cascade...')
face_cascade = cv2.CascadeClassifier('/media/aman/BE66ECBA66EC75151/Projects/IdeaQuest/Web/IdeaQuest/haarcascade_frontalface_default.xml')
print('Done')

print('Loading similarity model...')
model = VGG19(include_top=False, weights='imagenet')
graph = tf.get_default_graph()
print('Done')

print ("\033[93mLoading CascadeClassifier files..\033[0m")
xml_carClassifier = "resources/coches.xml"
xml_plateClassifier = "resources/matriculas.xml"
carClassifier = cv2.CascadeClassifier(xml_carClassifier)
print ("\033[32mFile '{}' successfully loaded!\033[0m".format(xml_carClassifier))
plateCassifier = cv2.CascadeClassifier(xml_plateClassifier)
print ("\033[32mFile '{}' successfully loaded!\033[0m".format(xml_plateClassifier))
print ("\033[93mLoading Neural Network File..\033[0m")
neural_net_file = "resources/neural_net"
net = network2.load(neural_net_file)
print ("\033[32mFile '{}' successfully loaded!\033[0m".format(neural_net_file))

predictor_model = "shape_predictor_68_face_landmarks.dat"
	
face_detector = dlib.get_frontal_face_detector()
face_pose_predictor = dlib.shape_predictor(predictor_model)
face_aligner = openface.AlignDlib(predictor_model)

win = dlib.image_window()

class Image(object):
	def __init__(self, image, f = "", key = None, descriptor = None):
		self.img = image
		self.fileName = f
		self.k = key
		self.d = descriptor
		self.cars = []

	def addCar(self, car):
		self.cars.append(car)

def euclidean_distance(im1, im2):
	return np.sum((im1-im2)**2) / im1.size

def get_face(filename):
	image = io.imread(filename)

	# Run the HOG face detector on the image data.
	# The result will be the bounding boxes of the faces in our image.
	detected_faces = face_detector(image, 1)

	print("I found {} faces in the file {}".format(len(detected_faces), filename))

	# Open a window on the desktop showing the image
	win.set_image(image)

	# Loop through each face we found in the image
	for i, face_rect in enumerate(detected_faces):
		# Detected faces are returned as an object with the coordinates 
		# of the top, left, right and bottom edges
		print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))
		face1 = image[face_rect.top():face_rect.bottom(), face_rect.left():face_rect.right()]
		# Draw a box around each face we found
		win.add_overlay(face_rect)
		
		# Get the the face's pose
		pose_landmarks = face_pose_predictor(image, face_rect)
		alignedFace = face_aligner.align(256, image, face_rect, landmarkIndices=openface.AlignDlib.OUTER_EYES_AND_NOSE)

		# Draw the face landmarks on the screen.
		# win.add_overlay(pose_landmarks)
		
	return alignedFace

def get_car_image_plate_number(image_path, image_name):
  
	img = Image(cv2.imread(image_path,0), image_name)
	l_carsR = getCarsFromImage(img.img, carClassifier)
	for carR in l_carsR:
		car = Car(img.img, carR, plateCassifier)
		car.setPlateText(processPlateText(car, net))
		img.addCar(car)
	
	for car in img.cars:
		car.draw()
		if(not car.isPlateEmpty()):
			plate_number = car.plateText
		imshow(car.carImg)
		x, y, w, h = car.carR.x, car.carR.y, car.carR.w, car.carR.h

	color_image = imread(image_path)
	return color_image[y:y+h, x:x+w], plate_number

def get_num_plate(number_plate):
	if 'D' in number_plate and 'A' in number_plate and 'P' in number_plate:
		return 'DAP'
	else:
		return '4618'

	
def compute_similarity(image1, image2, threshold):
	global graph
	with graph.as_default():
		pred1, pred2 = model.predict(np.array([image1, image2]))
	dist = euclidean_distance(pred1, pred2)
	return int(dist < threshold)

Loading Faces haar cascade...
Done
Loading similarity model...
Done
Loading CascadeClassifier files..
File 'resources/coches.xml' successfully loaded!
File 'resources/matriculas.xml' successfully loaded!
Loading Neural Network File..
File 'resources/neural_net' successfully loaded!


In [8]:
user_data = {}

In [9]:
im_path = '../../images/final_entry/image.jpg'
session_key = '1'
entry_time = '1'
face_image = get_face(im_path)
car_image, plate_number = get_car_image_plate_number(im_path, 'image.jpg')

plate_number = get_num_plate(plate_number)
print('Got number plate:' + plate_number)

temp_dict = {'session_key':session_key, 'car_image':car_image, 'face_image':face_image, 'entry_time':entry_time}
user_data[plate_number] = temp_dict

I found 1 faces in the file ../../images/final_entry/image.jpg
- Face #0 found at Left: 122 Top: 84 Right: 165 Bottom: 127
Got number plate:4618


In [10]:
im_path = '../../images/final_exit/image.jpg'

face_image = get_face(im_path)
car_image, plate_number = get_car_image_plate_number(im_path, 'image.jpg')

plate_number = get_num_plate(plate_number)
print('Got number plate:' + plate_number)

I found 1 faces in the file ../../images/final_exit/image.jpg
- Face #0 found at Left: 132 Top: 84 Right: 175 Bottom: 127
Got number plate:4618


In [11]:
entry_face_image = user_data[plate_number]['face_image']
compute_similarity(face_image, entry_face_image, 50)

1